# Notebook6: Geo-Exploration of my data

## Getting every year into the same address format


I realized in my 2017 exploration that it would be better to go through the geo-mapping in a different notebook since each year's data frames have different elements of the address given. Thus, it would be clearer to grapple with the entirity of their discrepancies in a different notebook.

# ISSUE SO FAR: CODE IS TOO LONG THAT IT ERRORS W/ ALL INCONSISTENCIES -- THIS NOTEBOOK WILL MOST LIKELY GO INTO ARCHIVE

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt  
import math
import os
import glob

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [33]:
#import energy usage data of Philadelphia's buildings from 2013-2017
yr2013_df = pd.read_csv('../data/energy_usage_large_commercial_buildings_reported_2013.csv')
yr2014_df = pd.read_csv('../data/energy_usage_large_commercial_buildings_reported_2014.csv')
yr2015_df = pd.read_csv('../data/properties_reported_2015.csv')
yr2016_df = pd.read_csv('../data/properties_reported_2016.csv')
yr2017_df = pd.read_csv('../data/properties_reported_2017.csv')

## Looking at the headers to see what form of address each dataframe has

In [4]:
yr2017_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,2310408.2,3589,1100 CATHARINE ST,Palumbo,781043800,19147,1,1980,K-12 School,185206.0,3330829,407768.5,4970622.0,NaN,97.0,41.5,68.6,672.8,646.8,NaN
1,23297409.0,4560,4100 CITY AVE,WPVI-TV,883080200,19131,1,2009,Office,69698.0,3681504,1242178.2,NaN,NaN,1.0,352.1,1068.3,2322.0,2650.4,NaN
2,NaN,4573,4343 WHITAKER AVE,ECI Building,884184500,19124,1,1950,Distribution Center,93000.0,3956329,663312.3,NaN,NaN,100.0,7.1,7.5,68.1,35.2,NaN
3,4749439.2,4153,3801 MARKET ST,First District Plaza,883805600,19104,1,1986,Other,87830.0,6432105,432235.7,NaN,NaN,NaN,59.3,175.9,1838.1,549.8,NaN
4,1168752.3,3450,1100 TYSON AVE,Moore,786543400,19111,1,1980,K-12 School,67701.0,3490894,3762754.7,NaN,NaN,32.0,72.8,112.6,1456.4,329.5,NaN


2017 has the street address and postal code 

In [5]:
yr2016_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,22205732.7,5260,233-43 S BROAD ST,Doubletree Hotel,883702001,19107,1,1983,Hotel,565764.0,3774262,10622801.8,NaN,NaN,78.0,58.0,143.0,23640.1,3110.8,NaN
1,1160875.0,6195,700-50 CATHEDRAL RD,Cathedral East Apartments,881088500,19128,1,1964,Multifamily Housing,92156.0,5140660,4181930.1,NaN,NaN,84.0,58.0,87.2,2190.6,355.3,NaN
2,680100.7,6119,8020 ALGON AVE,8020 Algon,881183500,19152,1,1964,Multifamily Housing,50840.0,5029250,1824667.9,NaN,NaN,95.0,49.3,79.7,1021.8,174.9,NaN
3,1564263.5,4993,926-76 E CHELTEN AVE,Pastorius,785099400,19138,1,1967,K-12 School,75318.0,3337464,2648097.0,NaN,NaN,86.0,55.9,102.1,3146.3,320.0,NaN
4,584826.7,5327,6951 NEW STATE RD,MARJAM SUPPLY,884182000,19135,1,1945,Distribution Center,106400.0,3813146,467121.4,NaN,NaN,91.0,9.9,21.9,612.2,91.9,NaN


2016 has the street address and postal code 

In [6]:
yr2015_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,4192843.0,2806,4501 BENNER ST,Lawton,786033600,19135,1,1973,K-12 School,79856.0,3330959,173627.5,NaN,NaN,37.0,54.7,167.1,870.9,570.5,NaN
1,1061172.9,3551,500-50 E ERIE AVE,500 Erie Ave,884048000,19134,1,1930,Distribution Center,130000.0,4104348,NaN,NaN,NaN,86.0,8.2,25.6,154.4,142.1,NaN
2,35489338.0,2841,3701 MARKET ST,3701 Market St.,883073300,19104,1,2000,Medical Office,140913.0,3340008,NaN,NaN,NaN,1.0,251.9,790.8,6528.2,4750.8,NaN
3,4934315.0,3229,1600-46 RACE ST,Friends Select School,772199000,19103,1,1969,K-12 School,123306.0,3891486,6447756.2,NaN,NaN,39.0,92.3,180.6,1083.3,1003.0,NaN
4,6759973.8,3103,14700 TOWNSEND RD,NARA,884292516,19154,1,1994,Other - Public Services,300000.0,3790331,4453888.7,NaN,NaN,NaN,37.4,86.3,103.9,1141.5,NaN


2015 has street address and postal code

In [7]:
yr2014_df.head()

,natural_gas_use_kbtu,portfolio_manager_id,postal_code,property_name,source_eui_kbtu_ft,total_ghg_emissions_mtco2e,district_steam_use_kbtu,site_eui_kbtu_ft,primary_property_type_epa_calculated,number_of_buildings,energy_star_score,location_1_address,year_built,fuel_oil_2_use_kbtu,property_floor_area_buildings_and_parking_ft,water_use_all_water_sources_kgal,electricity_use_kbtu,notes,philadelphia_building_id
0,"5,108,484.20",3330960,19120.0,Lowell,245.2,651.0,Not Available,136.5,K-12 School,1,11,450 W NEDRO AVE,1914,Not Available,58200.0,1493.9,2836085.4,NaN,785548000.0
1,"6,566,999.70",3952012,19116.0,10814 NE AVENUE,35.6,715.1,Not Available,21.4,Other - Services,1,Not Available,10814 NE AVENUE,1979,Not Available,217560.0,245.5,2736515.4,NaN,884240500.0
2,"243,912.00",3989966,19124.0,Juniata,34.1,88.9,Not Available,13.6,Self-Storage Facility,1,Not Available,"4123 ""G"" St.",2005,Not Available,59850.0,19.7,567623.4,NaN,884114205.0
3,"5,401,817.60",3330939,19143.0,Dobson,146.5,373.0,Not Available,115.1,K-12 School,1,12,4667 UMBRIA,1930,Not Available,52500.0,379.5,643237.4,NaN,784463600.0
4,"78,090.40",3330927,19128.0,Levering,121.5,546.1,Not Available,97.6,K-12 School,1,54,6000 RIDGE AVE,1929,"5,955,528.10",69475.0,284.1,747159.8,NaN,784378000.0


2014 has postal code and location address 

In [8]:
yr2013_df.head()

,the_geom,lng,year_built,electricity_use_grid_purchase_and_generated_from_onsite_renewab,energy_star_score,portfolio_manager_id,water_use_all_water_sources_kgal,district_steam_use_kbtu,site_eui_kbtu_ft2,philadelphia_building_id,...,fuel_oil_2_use_kbtu,number_of_buildings,property_floor_area_buildings_and_parking_ft,postal_code,natural_gas_use_kbtu,notes,total_ghg_emissions_mtco2e,primary_property_type_epa_calculated,lat,source_eui_kbtu_ft2
0,0101000020E61000002CBA8FB8D4CA52C0C82FAB660CFE...,-75.169233,1940,2146869.3,25,3330868,1604.5,Not Available,94.0,784321800.0,...,Not Available,1,109651.0,19121.0,8157192.2,NaN,704.8,K-12 School,39.984753,139.6
1,0101000020E610000080FC0401C6CB52C0A0BF7A90E9F9...,-75.183960,1917,3916010.4,73,3443035,2900.9,1299111.2,67.8,773717200.0,...,Not Available,1,79200.0,19104.0,155304.3,NaN,619.1,Office,39.952440,177.0
2,0101000020E6100000DC15D69AE1CA52C068520C2805F9...,-75.170020,1905,13312010.1,1,3512105,262.0,Not Available,225.2,772614900.0,...,0,1,59101.0,19146.0,Not Available,NaN,1685.8,Office,39.945470,707.3
3,0101000020E6100000B09CFBE0B9BF52C0F07C5E429C0E...,-74.995720,1950,192780.4,100,3892836,28.4,Not Available,3.0,884233835.0,...,Not Available,1,64638.0,19116.0,0,NaN,24.4,Office,40.114144,9.4
4,0101000020E610000054D00AC543CA52C09851DFDEA7F9...,-75.160386,1925,2756896.0,44,3759390,985.2,558808.5,69.5,883311000.0,...,Not Available,1,51676.0,19107.0,274307.8,NaN,413.2,Office,39.950436,186.1


2013 has long, postal code, and lat 

# Putting every year's addresses in the same format 

## Starting with 2017
First I want to start by looking at 2017.

In [9]:
yr2017_df.head()

,electricity_use_kbtu,objectid,street_address,property_name,opa_account_num,postal_code,num_of_buildings,year_built,primary_prop_type_epa_calc,total_floor_area_bld_pk_ft2,portfolio_manager_id,natural_gas_use_kbtu,fuel_oil_o2_use_kbtu,steam_use_kbtu,energy_star_score,site_eui_kbtuft2,source_eui_kbtuft2,water_use_all_kgal,total_ghg_emissions_mtco2e,notes
0,2310408.2,3589,1100 CATHARINE ST,Palumbo,781043800,19147,1,1980,K-12 School,185206.0,3330829,407768.5,4970622.0,NaN,97.0,41.5,68.6,672.8,646.8,NaN
1,23297409.0,4560,4100 CITY AVE,WPVI-TV,883080200,19131,1,2009,Office,69698.0,3681504,1242178.2,NaN,NaN,1.0,352.1,1068.3,2322.0,2650.4,NaN
2,NaN,4573,4343 WHITAKER AVE,ECI Building,884184500,19124,1,1950,Distribution Center,93000.0,3956329,663312.3,NaN,NaN,100.0,7.1,7.5,68.1,35.2,NaN
3,4749439.2,4153,3801 MARKET ST,First District Plaza,883805600,19104,1,1986,Other,87830.0,6432105,432235.7,NaN,NaN,NaN,59.3,175.9,1838.1,549.8,NaN
4,1168752.3,3450,1100 TYSON AVE,Moore,786543400,19111,1,1980,K-12 School,67701.0,3490894,3762754.7,NaN,NaN,32.0,72.8,112.6,1456.4,329.5,NaN


In [10]:

geolocator = Nominatim(user_agent="application")
lookup = RateLimiter(geolocator.geocode, min_delay_seconds=1)

addresses=[]
for  street, zip in yr2017_df[['street_address','postal_code']].values.tolist():
     address = "{}, PA {}".format(street.title(), zip)
     print(address)
     location = lookup(address, language='en', exactly_one=True)
     print(location.latitude, location.longitude)

1100 Catharine St, PA 19147
39.940224 -75.162107
4100 City Ave, PA 19131
40.004934 -75.218439
4343 Whitaker Ave, PA 19124
40.0149096861288 -75.117932639593
3801 Market St, PA 19104
39.9571117 -75.1986006733615
1100 Tyson Ave, PA 19111
40.050666 -75.077463
1660 Comly St, PA 19149
40.013814 -75.055942
5600-54 Vine St, PA 19139


AttributeError: 'NoneType' object has no attribute 'latitude'

Since the above code ran into error, we can make a better functioning code that spits back when we don't have theh necessary information.

In [11]:
data=[]
for street, zip in yr2017_df[['street_address','postal_code']].values.tolist():
    address = "{}, PA {}".format(street.title(), zip)
    print(address)
    location = lookup(address, language='en', exactly_one=True)
    try:
         data.append({'address': address,
                      'full_address': location.address,
                      'longitude': location.longitude,
                      'latitude': location.latitude
                     })
    except:
         data.append({'address': address})


1100 Catharine St, PA 19147
4100 City Ave, PA 19131
4343 Whitaker Ave, PA 19124
3801 Market St, PA 19104
1100 Tyson Ave, PA 19111
1660 Comly St, PA 19149
5600-54 Vine St, PA 19139
2600-20 N Broad St, PA 19132
3501 Cedar St, PA 19134
11550 E Roosevelt Blvd, PA 19116
9501 State Rd, PA 19114
1700 Bigler St, PA 19145
1330 Vernon Rd, PA 19150
3525 I St, PA 19134
1301-19 Filbert St, PA 19107
215 S Broad St, PA 19107


RateLimiter caught an error, retrying (0/2 tries). Called with (*('215 S Broad St, PA 19107',), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1321

20 S 36Th St, PA 19104
2213-21 Chestnut St, PA 19103
1645 Spring Garden St, PA 19130
620-34 N 15Th St, PA 19130
2200-24 Chestnut St, PA 19103
3401-15 Market St, PA 19104
1901-05 Walnut St, PA 19103
1516-18 Walnut St, PA 19102
1207-09 Chestnut St, PA 19107
233-55 S 8Th St, PA 19106
1429-33 Walnut St, PA 19102
201 N 34Th St, PA 19104
1015 Walnut St, PA 19107-5099
1026-44 Market St, PA 19107-4205
1501-11 Race St, PA 19102
201 N 17Th St, PA 19103


RateLimiter caught an error, retrying (0/2 tries). Called with (*('201 N 17Th St, PA 19103',), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1321,

1001-41 Walnut St, PA 19107-5244
1616-26 Walnut St, PA 19103
3501 Chestnut St, PA 19104
400-26 S Broad St, PA 19146
3711L Market St, PA 19104
1200 Market St, PA 19107
1500-06 Walnut St, PA 19102
101 N Independence Mall E, PA 19106
130 S 18Th St, PA 19103
1025 Walnut St, PA 19107-4083
1319-25 Market St, PA 19107
2000-24 Market St, PA 19103
1512-14 Spruce St, PA 19102
23-31 N Juniper St, PA 19107
1301-25 Chestnut St, PA 19107
1201 Market St, PA 19107
1000-32 Chestnut St, PA 19107-4998
111 South11Th St, PA 19107-5098
1701 John F Kennedy Blvd, PA 19103
801-49 Pine St, PA 19104
1400 S Penn Sq, PA 19102
3800 Powelton Ave, PA 19104
3401-41 N Broad St, PA 19140
1313-17 Walnut St, PA 19107
3400 Spruce St, PA 19104
230-34 W Washington Sq, PA 19107
122-30 S 9Th St, PA 19107-5233
1701-15 Locust St, PA 19103
17 S 17Th St, PA 19103


...

2135 E Westmoreland St, PA 19134
400-18 N 5Th St, PA 19123
7800 Bustleton Ave, PA 19152
1735 Market St, PA 19103
5213 Grays Ave, PA 19143
2233-47 W Allegheny Ave, PA 19132
3943-45 Chestnut St, PA 19104
5500 City Ave, PA 19131
7107 Old York Rd, PA 19126
211 S Christopher Columbus Blvd, PA 19106
2930 Chestnut St, PA 19104
222 N 17Th St, PA 19103


RateLimiter caught an error, retrying (0/2 tries). Called with (*('222 N 17Th St, PA 19103',), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/geopy/geocoders/base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1320, in do_open
    r = h.getresponse()
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1321,

1210-16 W Godfrey Ave, PA 19141
2127 Gillingham St, PA 19124
1511-15 Spruce St, PA 19102
1700 Spring Garden St, PA 19130


RateLimiter caught an error, retrying (0/2 tries). Called with (*('1700 Spring Garden St, PA 19130',), **{'language': 'en', 'exactly_one': True}).
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/usr/local/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/usr/local/anaconda3

6701-41 Callowhill St, PA 19151
209-25 N 18Th St, PA 19103
3301 S 20Th St, PA 19145
9250 Ashton Rd, PA 19114
7700-80 Crittenden St, PA 19118
3039-51 B St, PA 19134
4701-15 Kingsessing Ave, PA 19143
3939-41 Chestnut St, PA 19104
1700-20 Benjamin Franklin Pkwy, PA 19103
1751 Callowhill St, PA 19130
9455-73 E Roosevelt Blvd, PA 19114
13500 E Roosevelt Blvd, PA 19116
1331 E Wyoming Ave, PA 19124
4415 Aramingo Ave, PA 19124
1148 Wharton St, PA 19147
726-28 Market St, PA 19106
1701-33 W Girard Ave, PA 19130


In [ ]:
geo2017_df=pd.DataFrame(data)

In [ ]:
geo2017_df.shape

Because this dataframe is so large and takes a long time to load, we used a sample to show what to do

In [ ]:
sample2017=yr2017_df.head(39)

In [ ]:
pd.concat?

## 2016

In [ ]:
yr2016_df.head()

In [ ]:
geolocator = Nominatim(user_agent="application")
lookup = RateLimiter(geolocator.geocode, min_delay_seconds=1)

addresses=[]
for  street, zip in yr2016_df[['street_address','postal_code']].values.tolist():
     address = "{}, PA {}".format(street.title(), zip)
     print(address)
     location = lookup(address, language='en', exactly_one=True)
     print(location.latitude, location.longitude)

In [ ]:
data=[]
for street, zip in yr2016_df[['street_address','postal_code']].values.tolist():
    address = "{}, PA {}".format(street.title(), zip)
    print(address)
    location = lookup(address, language='en', exactly_one=True)
    try:
         data.append({'address': address,
                      'full_address': location.address,
                      'longitude': location.longitude,
                      'latitude': location.latitude
                     })
    except:
         data.append({'address': address})



In [ ]:
geo2016_df=pd.DataFrame(data)

In [ ]:
geo2016_df.shape

In [ ]:
sample2016=yr2016_df.head(39)

In [ ]:
pd.concat([sample2016, geo2016_df], axis=1)

In [ ]:
pd.concat?

## 2015

In [ ]:
yr2015_df.head()

In [ ]:
geolocator = Nominatim(user_agent="application")
lookup = RateLimiter(geolocator.geocode, min_delay_seconds=1)

addresses=[]
for  street, zip in yr2015_df[['street_address','postal_code']].values.tolist():
     address = "{}, PA {}".format(street.title(), zip)
     print(address)
     location = lookup(address, language='en', exactly_one=True)
     print(location.latitude, location.longitude)

In [ ]:
data=[]
for street, zip in yr2015_df[['street_address','postal_code']].values.tolist():
    address = "{}, PA {}".format(street.title(), zip)
    print(address)
    location = lookup(address, language='en', exactly_one=True)
    try:
         data.append({'address': address,
                      'full_address': location.address,
                      'longitude': location.longitude,
                      'latitude': location.latitude
                     })
    except:
         data.append({'address': address})



In [ ]:
geo2015_df=pd.DataFrame(data)

In [ ]:
geo2015_df.shape

In [ ]:
sample2015=yr2015_df.head(39)

In [ ]:
pd.concat([sample2015, geo2015_df], axis=1)

## 2014

In [ ]:
yr2014_df.head()

Here we see that the address is titled something different than the previous years as "location_1_address". So now in a similar fashion of above, I will get my dataframe ready but by replacing "street_address" with "location_1_address" 

In [ ]:
geolocator = Nominatim(user_agent="application")
lookup = RateLimiter(geolocator.geocode, min_delay_seconds=1)

addresses=[]
for  street, zip in yr2014_df[['location_1_address','postal_code']].values.tolist():
     address = "{}, PA {}".format(street.title(), zip)
     print(address)
     location = lookup(address, language='en', exactly_one=True)
     print(location.latitude, location.longitude)

In [ ]:
data=[]
for street, zip in yr2014_df[['location_1_address','postal_code']].values.tolist():
    address = "{}, PA {}".format(street.title(), zip)
    print(address)
    location = lookup(address, language='en', exactly_one=True)
    try:
         data.append({'address': address,
                      'full_address': location.address,
                      'longitude': location.longitude,
                      'latitude': location.latitude
                     })
    except:
         data.append({'address': address})


In [ ]:
geo2014_df=pd.DataFrame(data)

In [ ]:
geo2014_df.shape

In [ ]:
sample2014=yr2014_df.head(39)

In [ ]:
pd.concat([sample2014, geo2014_df], axis=1)

## 2013

In [ ]:
yr2013_df.head()

2013 is a very different dataframe. It gives the longitude, latittude, and post code. So now, in an opposite fashion of above, I need to reverse lookup. 

In [ ]:
geolocator = Nominatim(user_agent="application")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

addresses=[]
for long,lat in yr2013_df[['lng','lat']].values.tolist():
     location = reverse((lat,long), language='en', exactly_one=True)
     print(long, lat, location)
     addresses.append(location.address)

In [ ]:
yr2013_df['street_address']=addresses

## Putting everything on a map now

In [ ]:
map = [point(xy) in yr2013_df['lng'], yr2017_Df['lat']

In [ ]:
yr2013_df.plot()